In [1]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1])


/home/xilun/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-06-13 18:20:39,826] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/xilun/miniconda3/envs/et_robot/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like meta-llama/Meta-Llama-3-70B-Instruct is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [2]:
import os 
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig
import json

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.random.manual_seed(0)

# Ensure the device is set correctly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the directory where the fine-tuned LoRA adapters are saved
output_dir = "phi-3-mini-LoRA/checkpoint-1097/"

model_name = "microsoft/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, add_eos_token=True, use_fast=True)


# Load the LoRA adapter configuration
peft_config = PeftConfig.from_pretrained(output_dir)

# Apply the LoRA adapter to the base model
# model = PeftModel.from_pretrained(model, output_dir, torch_dtype=torch.bfloat16)
# model = model.merge_and_unload()
model = model.to(device)

# prompt_path = '/home/xilun/ET_robot/prompt_stack_two.txt'
prompt_path = "/home/xilun/ET_robot/dataset/172_dataset.json"
# Load the prompt as json file
prompt = json.loads(open(prompt_path).read())
prompt = prompt[1]["input"]
## delete <|user|> and <|assistant|> and <|end|> if found in prompt 
# prompt = prompt.replace("<|user|>", "").replace("<|assistant|>", "").replace("<|end|>", "")

# inputs = tokenizer(prompt, return_tensors="pt").to(device)
# prompt = prompt + "Please generate the plan"
messages = [
    {"role": "user", "content": prompt},
]

# Generate text using the model's generate method
generation_args = {
    "max_new_tokens": 1500,
    "temperature": 0.0,
    "do_sample": False,
    "return_full_text": False,
    # "top_k": 50,
    # "top_p": 0.95
    
}

# Generate the output
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])



# output = model.generate(
#     input_ids=inputs["input_ids"],
#     max_new_tokens=generation_args["max_new_tokens"],
#     temperature=generation_args["temperature"],
#     do_sample=generation_args["do_sample"],
#     # top_k=generation_args["top_k"],
#     # top_p=generation_args["top_p"],
#     # pad_token_id=tokenizer.pad_token_id,
#     # eos_token_id=tokenizer.eos_token_id,
# )
# # Decode the generated output with chat template



# output_text = tokenizer.decode(output[0], skip_special_tokens=True)
# print(output_text)
del model

# 'import gc' is used to import Python's garbage collector module.
import gc

# 'gc.collect()' is a method that triggers a full garbage collection, which can help to free up memory.
# It's called twice here to ensure that all unreachable objects are collected.
gc.collect()
gc.collect()
torch.cuda.empty_cache()


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.21it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 ```python

import numpy as np


# Step 1: Open the gripper to prepare for picking up the ball

open_gripper()


# Step 2: Move to the position above CubeC to avoid pushing it away

cubeC_position = get_position('cubeC')  # Assuming get_position() returns the position of CubeC

move_to_position(cubeC_position + np.array([0, cubeC_position[1], cubeC_position[2]]))


# Step 3: Get the graspable point of BallA

ballA_graspable_point = get_graspable_point('ballA')


# Step 4: Close the gripper to grasp BallA

close_gripper()


# Step 5: Move to the position above CubeC where BallA will be placed

move_to_position(cubeC_position + np.array([0, cubeC_position[1], cubeC_position[2] + ballA_graspable_point[2] - cubeC_size[2] / 2]))


# Step 6: Open the gripper to release BallA onto CubeC

open_gripper()

```
